`vdom` is Python data object for composing HTML. `mistletoe` is a markdown converter; in this notebook we write a `mistletoe` to `vdom`
renderer.

> There are some outstanding issues about how to handle inline span html.

> __Personal Opinion__: `mistletoe` is easier to extend than `mistune` and `commonmark`.

`vdom`, like `altair`, embed extra data into the notebook that can be consumed later.

This post builds [on previous work that converts `html` to `vdom`](2018-11-01-vdom-from-raw-html.ipynb).

In [1]:
from deathbeds import __vdom_from_raw_html  as vdom_raw
import mistletoe, vdom, functools, html

`vdomRenderer` replaces all of the `mistletoe.html_renderer.HTMLRenderer` methods to return `vdom` objects.  It was possible to simplify
most of the original `mistletoe.html_renderer.HTMLRenderer` by returning `vdom`.

In [2]:
class vdomRenderer(mistletoe.html_renderer.HTMLRenderer):
    def render_inner(self, token, tag=None, **data):
        return tag(*(self.render(child) for child in token.children), **data)

    render_document = functools.partialmethod(render_inner, tag=vdom.div)
    render_strong = functools.partialmethod(render_inner, tag=vdom.strong)
    render_emphasis = functools.partialmethod(render_inner, tag=vdom.em)
    render_strikethrough = functools.partialmethod(render_inner, tag=vdom.create_component('del'))        
    render_escape_sequence = functools.partialmethod(render_inner, tag=vdom.span)
    render_paragraph = functools.partialmethod(render_inner, tag=vdom.p)
    render_quote = functools.partialmethod(render_inner, tag=vdom.blockquote)
    render_quote = functools.partialmethod(render_inner, tag=vdom.blockquote)
    render_list_item = functools.partialmethod(render_inner, tag=vdom.li)

    def render_image(self, token):
        if token.title: title = self.escape_html(token.title)
        else: title = ''            
        return vdom.img(src=token.src, alt=self.render_to_plain(token), title=title)

    def render_to_plain(self, token):
        if hasattr(token, 'children'):
            inner = [self.render_to_plain(child) for child in token.children]
            return ''.join(inner)
        return self.escape_html(token.content)

    def render_link(self, token):
        target = self.escape_url(token.target)
        if token.title: title = self.escape_html(token.title)
        else: title = ''            
        return self.render_inner(token, vdom.a, href=target, title=title)

    def render_auto_link(self, token):
        if token.mailto:
            target = 'mailto:{}'.format(token.target)
        else:
            target = self.escape_url(token.target)
        return self.render_inner(token, vdom.a, href=target)

    def render_raw_text(self, token): return self.escape_html(token.content)

    def render_heading(self, token): return self.render_inner(token, getattr(vdom, F'h{token.level}'))

    def render_block_code(self, token):
        return vdom.pre(vdom.code(html.escape(token.children[0].content), **({
            'class': F"language-{self.escape_html(token.language)}"
        } if token.language else {})))

    def render_list(self, token, tag=vdom.ul):
        if token.start is not None:
            tag = vdom.ol
            if token.start != 1:
                tag = functools.partial(tag, start=str(token.start))
            attr = ' start="{}"'.format(token.start) if token.start != 1 else ''
        return self.render_inner(token, tag)

    def render_table(self, token, tag=vdom.table):
        if hasattr(token, 'header'):
            tag = functools.partial(tag,vdom.thead(self.render_table_row(token.header, is_header=True)))
        return tag(self.render_inner(token, vdom.tbody))

    def render_table_row(self, token, is_header=False):
        return vdom.tr(*(self.render_table_cell(child, is_header) for child in token.children))

    def render_table_cell(self, token, in_header=False):
        return self.render_inner(token, vdom.th if in_header else vdom.td, align={
            None: 'left', 0: 'center', 1: 'right'
        }[token.align])

    @staticmethod
    def render_thematic_break(token): return vdom.hr()

    @staticmethod
    def render_line_break(token): return '\n' if token.soft else vdom.br()

    @staticmethod
    def render_html_block(token): return vdom_raw.vdomParser()(token.content)

    def render_inline_code(self, token): 
        return self.render_inner(html.escape(token.children[0].content), vdom.code)

    _dom = None
    
    def render_html_span(self, token):  
        self._dom = (self._dom or vdom_raw.vdomParser())
        self._dom.feed(token.content)
        return self._dom.buf[0]() if len(self._dom.buf) == 1 else ''
    

In [3]:
def test_it_runs():
    vdomRenderer().render(mistletoe.Document("""# A table 

    | Tables        | Are           | Cool  |
    | ------------- |:-------------:| -----:|
    | col 3 is      | right-aligned | $1600 |
    | col 2 is      | centered      |   $12 |
    | zebra stripes | are neat      |    $1 |
    
    * Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu 
        * > Does a block quote work in line item

      Then we talk more
        * Fuck
    * ###  B

    > Testing things

    ### Howdy\n\n~~Good~~ look buh <span><code>__crap__</code></span>.\n\n<h1>Things</h1>""".splitlines()))



In [15]:
    if __name__ == '__main__':
        !ipython -m pytest -- 2018-11-03-Markdown-to-vdom-with-mistletoe.ipynb

============================= test session starts =============================
platform win32 -- Python 3.6.6, pytest-3.5.1, py-1.5.3, pluggy-0.6.0
Matplotlib: 2.2.2
Freetype: 2.8.1
rootdir: C:\Users\deathbeds\deathbeds.github.io\deathbeds, inifile: tox.ini
plugins: xonsh-0.8.1, xdist-1.22.5, testmon-0.9.12, remotedata-0.2.1, parallel-0.0.2, openfiles-0.3.0, mpl-0.9, localserver-0.4.1, forked-0.2, doctestplus-0.1.3, arraydiff-0.2, hypothesis-3.66.16, importnb-0.5.0
collected 2 items

2018-11-03-Markdown-to-vdom-with-mistletoe.ipynb ..                      [100%]

========================== 2 passed in 0.41 seconds ===========================


In [14]:
    def test_graphviz_in_markdown_in_vdom():
        import graphviz
        g = graphviz.Source("graph {A}")
        vdomRenderer().render(mistletoe.Document(
        F"""# bUST

        {''.join(g._repr_svg_().splitlines())}""".splitlines()
        ))